In [55]:
import pandas_dq
import pandas as pd
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('datasets\iris.csv')
le = LabelEncoder()

# Fit the encoder to the Species column
le.fit(data['Species'])

# Transform the Species column using the fitted encoder
data['Species'] = le.transform(data['Species'])
report = pandas_dq.dq_report(data, target='Species', csv_engine="pandas", verbose=1)
reportDICT = report.to_dict()
df = pd.DataFrame(reportDICT)
print(df)
# reportdf = pd.DataFrame.from_dict(reportDICT)
# print(reportdf.head())
#
# #print(reportJSON)
# reportDICT = report.to_dict()
# reportdf = pd.DataFrame.from_dict(reportDICT)
# print(reportdf.head())
# with open('htmlcode.txt', 'w') as f:
#     f.write(reportDICT)
#print(reportDICT)

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Id,int64,0.000000,100,1.000000,150.000000,"Possible ID colum: drop before modeling process., Id has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable."
SepalLengthCm,float64,0.000000,NA,4.300000,7.900000,No issue
SepalWidthCm,float64,0.000000,NA,2.000000,4.400000,has 4 outliers greater than upper bound (4.05) or lower than lower bound(2.05). Cap them or remove them.
PetalLengthCm,float64,0.000000,NA,1.000000,6.900000,"has a high correlation with ['Id', 'SepalLengthCm']. Consider dropping one of them., PetalLengthCm has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable."
PetalWidthCm,float64,0.000000,NA,0.100000,2.500000,"has a high correlation with ['Id', 'SepalLengthCm', 'PetalLengthCm']. Consider dropping one of them., PetalWidthCm has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable."
Species,int32,0.000000,2,0.000000,2.000000,"has a high correlation with ['Id', 'PetalLengthCm', 'PetalWidthCm']. Consider dropping one of them."


              Data Type  Missing Values% Unique Values%  Minimum Value  \
Id             int64     0.0              100            1.0             
SepalLengthCm  float64   0.0              NA             4.3             
SepalWidthCm   float64   0.0              NA             2.0             
PetalLengthCm  float64   0.0              NA             1.0             
PetalWidthCm   float64   0.0              NA             0.1             
Species        int32     0.0              2              0.0             

               Maximum Value  \
Id             150.0           
SepalLengthCm  7.9             
SepalWidthCm   4.4             
PetalLengthCm  6.9             
PetalWidthCm   2.5             
Species        2.0             

                                                                                                                                                                                                                       DQ Issue  
Id             Possible ID col

In [56]:
from pandas_dq import Fix_DQ
fdq = Fix_DQ()

X_train_transformed = fdq.fit_transform(data)


1 ID cols will be dropped from further processing: ['Id']
    Dropping PetalLengthCm which has a high correlation with ['Id', 'SepalLengthCm']
    Dropping PetalWidthCm which has a high correlation with ['Id', 'SepalLengthCm', 'PetalLengthCm']
    Dropping Species which has a high correlation with ['Id', 'PetalLengthCm', 'PetalWidthCm']


In [57]:
report = pandas_dq.dq_report(X_train_transformed, target=None, csv_engine="pandas", verbose=1)

Alert: Dropping 34 duplicate rows can sometimes cause column data types to change to object. Double-check!


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
SepalLengthCm,float64,0.000000,NA,4.300000,7.900000,No issue
SepalWidthCm,float64,0.000000,NA,2.000000,4.050000,has 1 outliers greater than upper bound (4.05) or lower than lower bound(2.05). Cap them or remove them.


In [58]:
import cleanlab
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv('datasets\Iris.csv')
# Create a label encoder object
le = LabelEncoder()
# Fit and transform the "Species" column
df['Species'] = le.fit_transform(df['Species'])
def cleanlab_test(df):

    # cleanlab works with **any classifier**. Yup, you can use PyTorch/TensorFlow/OpenAI/XGBoost/etc.
    cl = cleanlab.classification.CleanLearning(RandomForestClassifier())
    data = df.loc[:, df.columns != 'class']

    # cleanlab finds data and label issues in **any dataset**... in ONE line of code!
    # Fit model to messy, real-world data, automatically training on cleaned data.
    _ = cl.fit(data, df['class'])
    # See the label quality for every example, which data has issues, and more.
    print(cl.get_label_issues())

In [59]:
from deepchecks.tabular import Dataset
import deepchecks.tabular.checks
import pandas as pd
from sklearn.datasets import load_iris
from deepchecks.tabular.datasets.classification.phishing import load_data
import numpy as np
from deepchecks.tabular.datasets.classification import adult
import plotly.express as px
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

deepchecks - WARNING - You are using deepchecks version 0.11.1, however a newer version is available.Deepchecks is frequently updated with major improvements. You should consider upgrading via the "python -m pip install --upgrade deepchecks" command.


In [60]:
def encode_categorical_columns(dataset, target, data_types):
    # Find all categorical columns
    # TODO: regel hieronder aanpassen naar wat de user heeft ingegeven (let hierbij op strings als dtype, moeten gezien worden als categorical)
    categorical_cols = dataset.select_dtypes(include=['object', 'category']).columns.tolist()

    target_is_categorical = False
    if target != 'None':
        # remove target as we want to label encode this (for classification problems)
        if target in categorical_cols:
            target_is_categorical = True
            categorical_cols.remove(target)

            # label encode target
            le = LabelEncoder()
            encoded_target = le.fit_transform(dataset[target])
            # replace target column with label encoded values
            dataset.drop(columns=[target], inplace=True)
            dataset[target] = encoded_target

    if not categorical_cols:  # then no features are categorical, and we're done
        return dataset

    # if there are categorical columns, we want to one-hot-encode them

    # encode categoricals
    encoder = OneHotEncoder(handle_unknown='ignore', max_categories=100)
    encoded_columns = encoder.fit_transform(dataset[categorical_cols])
    new_columns = pd.DataFrame(encoded_columns.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

    # add new columns to df and drop old ones
    dataset_encoded = pd.concat([dataset, new_columns], axis=1)
    dataset_encoded = dataset_encoded.drop(columns=categorical_cols)

    # reposition target column to the end of the dataframe
    if target != 'None' and target_is_categorical:
        dataset_encoded.drop(columns=[target], inplace=True)
        dataset_encoded[target] = encoded_target

    #XGBClassifier doesn't accept: [, ] or <, so loop over the columns and change the names if they contain such values
    new_col_names = {col: col.replace('<', '(smaller than)').replace('[', '(').replace(']', ')') for col in dataset_encoded.columns}
    dataset_encoded = dataset_encoded.rename(columns=new_col_names)

    return dataset_encoded

In [61]:
def wrong_label(encoded_dataset, target):
    """"Function that finds potential label errors (due to annotator mistakes), edge cases, and otherwise ambiguous examples"""
    model_XGBC = XGBClassifier(tree_method="hist", enable_categorical=True) #hist is fastest tree method of XGBoost
    #TODO: add more models and choose whichever gives highest prediction? Downside --> slows process
    data_no_labels = encoded_dataset.drop(columns=[target])
    labels = encoded_dataset[target]
    pred_probs = cross_val_predict(model_XGBC, data_no_labels, labels, method='predict_proba')

    preds = np.argmax(pred_probs, axis=1)
    acc_original = accuracy_score(preds, labels)
    print(f"Accuracy with original data: {round(acc_original * 100, 1)}%")

    #use cleanlab's built in confident learning method to find label issues
    cl = cleanlab.classification.CleanLearning()
    issues_dataframe = cl.find_label_issues(X=None, labels=labels, pred_probs=pred_probs)
    wrong_label_count = (issues_dataframe['is_label_issue'] == True).sum()

    #filter df so only errors are visible
    issues_dataframe_only_errors = issues_dataframe[issues_dataframe['is_label_issue'] == True]

    return issues_dataframe, issues_dataframe_only_errors, wrong_label_count

In [62]:
from scipy.io.arff import loadarff

raw_data = loadarff('datasets/dataset_31_credit-g.arff')
df_data = pd.DataFrame(raw_data[0])
#df_data.drop(columns=['checking_status', 'savings_status', 'employment'], inplace=True)
encdoded_data = encode_categorical_columns(df_data, 'class', None)
# pred_probs, labels = wrong_label(encdoded_data, 'class')

wrong_label(encdoded_data, 'class')

# cl = cleanlab.classification.CleanLearning()
# issues_dataframe = cl.find_label_issues(X=None, labels=labels, pred_probs=pred_probs, return_indices_ranked_by='self_confidence')
# print(issues_dataframe)

Accuracy with original data: 75.7%


(     is_label_issue  label_quality  given_label  predicted_label
 0    False           0.982974       1            1              
 1    False           0.896697       0            0              
 2    False           0.995315       1            1              
 3    True            0.405909       1            0              
 4    False           0.830484       0            0              
 5    False           0.963891       1            1              
 6    False           0.997703       1            1              
 7    False           0.887332       1            1              
 8    False           0.999295       1            1              
 9    False           0.949440       0            0              
 10   False           0.933829       0            0              
 11   False           0.995936       0            0              
 12   False           0.600553       1            1              
 13   False           0.675812       0            0              
 14   True

In [63]:
import plotly.express as px
df = pd.read_csv('datasets/iris.csv')
print(df.head())


def pcp_plot(encoded_df, target):
    #TODO: tekst/lookup table toevoegen met conversie categorische variabelen encoding

    if target != 'None':
        fig = px.parallel_coordinates(encoded_df, color=target)
    else:
        fig = px.parallel_coordinates(encoded_df)


    return fig


df_encoded = encode_categorical_columns(df, 'target', None)
fig = px.parallel_coordinates(df_encoded, color='target')
fig.show()

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0  1   5.1            3.5           1.4            0.2           Iris-setosa
1  2   4.9            3.0           1.4            0.2           Iris-setosa
2  3   4.7            3.2           1.3            0.2           Iris-setosa
3  4   4.6            3.1           1.5            0.2           Iris-setosa
4  5   5.0            3.6           1.4            0.2           Iris-setosa


ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'] but received: target

In [ ]:
def encode_categorical_columns(dataset, target, data_types):
    # Find all categorical columns
    # TODO: regel hieronder aanpassen naar wat de user heeft ingegeven (let hierbij op strings als dtype, moeten gezien worden als categorical)
    categorical_cols = dataset.select_dtypes(include=['object', 'category']).columns.tolist()

    target_is_categorical = False
    if target != 'None':
        # remove target as we want to label encode this (for classification problems)
        if target in categorical_cols:
            target_is_categorical = True
            categorical_cols.remove(target)

            # label encode target
            le = LabelEncoder()
            encoded_target = le.fit_transform(dataset[target])
            # replace target column with label encoded values
            dataset.drop(columns=[target], inplace=True)
            dataset[target] = encoded_target

    if not categorical_cols:  # then no features are categorical, and we're done
        return dataset

    # if there are categorical columns, we want to one-hot-encode them

    # encode categoricals
    encoder = OneHotEncoder(handle_unknown='ignore', max_categories=100)
    encoded_columns = encoder.fit_transform(dataset[categorical_cols])
    new_columns = pd.DataFrame(encoded_columns.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

    # add new columns to df and drop old ones
    dataset_encoded = pd.concat([dataset, new_columns], axis=1)
    dataset_encoded = dataset_encoded.drop(columns=categorical_cols)

    # reposition target column to the end of the dataframe
    if target != 'None' and target_is_categorical:
        dataset_encoded.drop(columns=[target], inplace=True)
        dataset_encoded[target] = encoded_target

    #XGBClassifier doesn't accept: [, ] or <, so loop over the columns and change the names if they contain such values
    new_col_names = {col: col.replace('<', '(smaller than)').replace('[', '(').replace(']', ')') for col in dataset_encoded.columns}
    dataset_encoded = dataset_encoded.rename(columns=new_col_names)

    return dataset_encoded

In [ ]:
import missingno as msno

def missingno_plot(df):

    msno_plot = msno.matrix(df)

    return msno_plot


df = pd.read_csv('datasets/titanic_train.csv')
print(df.head())

missingno_plot(df)

In [ ]:
from sklearn.tree
def plot_feature_importance(X, y, feature_names):
    """
    Predicts the feature importance using a random forest and plots it.

    Parameters:
    -----------
    X : numpy array, shape (n_samples, n_features)
        The input data.
    y : numpy array, shape (n_samples,)
        The target values.
    feature_names : list of strings, shape (n_features,)
        The names of the features.

    Returns:
    --------
    None
    """
    # Initialize the random forest regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # Fit the random forest to the data
    rf.fit(X, y)

    # Get the feature importances
    feature_importances = rf.feature_importances_

    # Sort the feature importances in descending order
    indices = np.argsort(feature_importances)[::-1]

    # Plot the feature importances
    plt.figure()
    plt.title("Feature Importance")
    plt.bar(range(X.shape[1]), feature_importances[indices])
    plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
    plt.tight_layout()
    plt.show()




# Generate some random data
X = np.random.rand(100, 5)
y = np.random.rand(100)

# Define the feature names
feature_names = ["Feature 1", "Feature 2", "Feature 3", "Feature 4", "Feature 5"]

# Call the function to plot the feature importance
plot_feature_importance(X, y, feature_names)

In [ ]:
import sortinghatinf



In [ ]:
dict = {'Data Type': {'Id': 'dtype(int64)', 'SepalLengthCm': 'dtype(int64)', 'SepalWidthCm': 'dtype(int64)', 'PetalLengthCm': 'dtype(int64)', 'PetalWidthCm': 'dtype(int64)', 'Species': 'dtype(int64)'}, 'Missing Values%': {'Id': 0.0, 'SepalLengthCm': 0.0, 'SepalWidthCm': 0.0, 'PetalLengthCm': 0.0, 'PetalWidthCm': 0.0, 'Species': 0.0}, 'Unique Values%': {'Id': 100, 'SepalLengthCm': 'NA', 'SepalWidthCm': 'NA', 'PetalLengthCm': 'NA', 'PetalWidthCm': 'NA', 'Species': 2}, 'Minimum Value': {'Id': 1.0, 'SepalLengthCm': 4.3, 'SepalWidthCm': 2.0, 'PetalLengthCm': 1.0, 'PetalWidthCm': 0.1, 'Species': 0.0}, 'Maximum Value': {'Id': 150.0, 'SepalLengthCm': 7.9, 'SepalWidthCm': 4.4, 'PetalLengthCm': 6.9, 'PetalWidthCm': 2.5, 'Species': 2.0}, 'DQ Issue': {'Id': 'Possible ID colum: drop before modeling process.,     Id has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable.', 'SepalLengthCm': 'No issue', 'SepalWidthCm': 'has 4 outliers greater than upper bound (4.05) or lower than lower bound(2.05). Cap them or remove them.', 'PetalLengthCm': "has a high correlation with ['Id', 'SepalLengthCm']. Consider dropping one of them.,     PetalLengthCm has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable.", 'PetalWidthCm': "has a high correlation with ['Id', 'SepalLengthCm', 'PetalLengthCm']. Consider dropping one of them.,     PetalWidthCm has a correlation >= 0.8 with Species. Possible data leakage. Double check this variable.", 'Species': "has a high correlation with ['Id', 'PetalLengthCm', 'PetalWidthCm']. Consider dropping one of them."}}

import pandas as pd
from typing import Dict

def dict_to_dataframe(d: Dict[str, Dict[str, any]]) -> pd.DataFrame:
    data = {}
    for key in d:
        for sub_key, value in d[key].items():
            if sub_key not in data:
                data[sub_key] = []
            data[sub_key].append(value)
    return pd.DataFrame(data)
output = dict_to_dataframe(dict)
print(output.head())

In [128]:
import pandas as pd
from itertools import product
import numpy as np

def sensitive_feature_combinations(dataset_original, sensitive_features, target_column, bins=5):
    # Create a copy of the dataset to avoid modifying the original data
    dataset = dataset_original.copy()
    #TODO: callback maken die sensitive features ingeeft
    #TODO: iets bedenken voor regression
    #bin numeric sensitive features into 5 bins
    for feat in sensitive_features:
        if pd.api.types.is_numeric_dtype(dataset[feat]):        #TODO: baseren op data type inference sortinghat
            dataset[feat] = pd.cut(dataset[feat], bins=bins)

    #obtain all combinations of sensitive features
    sensitive_combinations = list(product(*[dataset[feat].unique() for feat in sensitive_features]))
    #print(sensitive_combinations)
    # Initialize empty dictionary to store counts
    counts = {}

    # Loop through each sensitive feature combination
    for combo in sensitive_combinations:
        # Create a mask for rows that match the current combination
        mask = True
        for i, feat in enumerate(sensitive_features):
            mask = mask & (dataset[feat] == combo[i])

        # Count the number of rows that match the mask and the target value
        count = dataset.loc[mask, target_column].value_counts()

        # Store the count in the dictionary
        counts[combo] = count.to_dict()

    # Convert dictionary to a pandas DataFrame
    result = pd.DataFrame.from_dict(counts, orient='index').fillna(0)

    # Add a column for the total count
    result['count'] = result.sum(axis=1)

    # Add a column for the sensitive feature combination
    result['sensitive_features'] = result.index.map(lambda x: ', '.join(map(str, x)))

    # Reorder the columns
    result = result[['sensitive_features', 'count'] + list(dataset[target_column].unique())]

    return result

In [129]:
data = pd.read_csv('datasets/adult.csv')
data.head()
sensitive_features = ['race', 'gender', 'age']
target_column = 'income'
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# # All dataframes hereafter reflect these changes.
# display(df)
#
# print('**RESET_OPTIONS**')
#
# # Resets the options
# pd.reset_option('all')

print(sensitive_feature_combinations(data, sensitive_features, target_column, bins=5))


C:\Users\robin\AppData\Local\Temp\ipykernel_21720\2858654096.py:9: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



                                                                  sensitive_features  \
Black              Male   (16.927, 31.6]  Black, Male, (16.927, 31.6]                  
                          (31.6, 46.2]    Black, Male, (31.6, 46.2]                    
                          (60.8, 75.4]    Black, Male, (60.8, 75.4]                    
                          (46.2, 60.8]    Black, Male, (46.2, 60.8]                    
                          (75.4, 90.0]    Black, Male, (75.4, 90.0]                    
                   Female (16.927, 31.6]  Black, Female, (16.927, 31.6]                
                          (31.6, 46.2]    Black, Female, (31.6, 46.2]                  
                          (60.8, 75.4]    Black, Female, (60.8, 75.4]                  
                          (46.2, 60.8]    Black, Female, (46.2, 60.8]                  
                          (75.4, 90.0]    Black, Female, (75.4, 90.0]                  
White              Male   (16.92

In [130]:
# Call the sensitive_feature_combinations function to get the counts table
counts_table = sensitive_feature_combinations(data, sensitive_features, target_column)


def plot_stacked_barchart(sensitive_feature_counts_table):
    """"creates a plotly stacked bar chart to display the distribution of class labels per sensitive subgroup"""
    #table always looks the same (sensitive features (vary in amount), count, class labels (vary in amount))
    list_of_labels = list(sensitive_feature_counts_table.columns)
    list_of_labels.remove('sensitive_features')
    list_of_labels.remove('count')
    fig = px.bar(sensitive_feature_counts_table, x=list_of_labels, y='sensitive_features', barmode='stack', title='Sensitive Feature Combinations vs Target Column', orientation='h')
    # Set the x-axis title
    fig.update_xaxes(title_text='Count')

    return fig

fig = plot_stacked_barchart(counts_table)
fig.show()


In [133]:
data = pd.read_csv('datasets/german_credit_data.csv')
data.head()
sensitive_features = ['Age', 'Sex', 'Job', 'Saving accounts']
target_column = 'Housing'
counts_table = sensitive_feature_combinations(data, sensitive_features, target_column)
print(counts_table)
fig = plot_stacked_barchart(counts_table)
fig.show()

                                                                               sensitive_features  \
(63.8, 75.0]   male   (1.8, 2.4]    little      (63.8, 75.0], male, (1.8, 2.4], little              
                      (0.6, 1.2]    little      (63.8, 75.0], male, (0.6, 1.2], little              
                                    quite rich  (63.8, 75.0], male, (0.6, 1.2], quite rich          
                      (2.4, 3.0]    little      (63.8, 75.0], male, (2.4, 3.0], little              
                      (-0.003, 0.6] little      (63.8, 75.0], male, (-0.003, 0.6], little           
               female (1.8, 2.4]    little      (63.8, 75.0], female, (1.8, 2.4], little            
                                    moderate    (63.8, 75.0], female, (1.8, 2.4], moderate          
                      (0.6, 1.2]    quite rich  (63.8, 75.0], female, (0.6, 1.2], quite rich        
                      (2.4, 3.0]    little      (63.8, 75.0], female, (2.4, 3.0], little   

In [136]:
import pandas as pd
import plotly.express as px

def plot_dataset_distributions(data, dtypes):
    """"plots the distributions per column in the dataset"""
    # Initialize an empty list to store the figures
    figs = []

    # Loop over each column in the dataset
    for col in data.columns:
        # Determine the type of the column
        col_type = data[col].dtype
        #tODO: fixen dtype inference sortinghatinf
        # If the column is numeric, create a histogram
        if pd.api.types.is_numeric_dtype(col_type):
            fig = px.histogram(data, x=col, title=f'{col} distribution')
            figs.append(fig)

        # If the column is boolean, create a bar chart
        elif pd.api.types.is_bool_dtype(col_type):
            fig = px.bar(data, x=col, color=col, title=f'{col} distribution')
            figs.append(fig)

        # If the column is categorical, create a bar chart
        elif pd.api.types.is_categorical_dtype(col_type):
            fig = px.bar(data, x=col, title=f'{col} distribution')
            figs.append(fig)

        # If the column is a string, create a word cloud
        elif pd.api.types.is_string_dtype(col_type):
            word_counts = data[col].str.split(expand=True).stack().value_counts()
            fig = px.bar(x=word_counts.index, y=word_counts.values, title=f'{col} word cloud')
            fig.update_layout(xaxis={'categoryorder':'total descending'})
            figs.append(fig)

    return figs

figs = plot_dataset_distributions(data, None)
for fig in figs:
    fig.show()

In [137]:
import sortinghatinf #algorithm to predict the feature type out og [numeric, categorical, datetime, sentence,
def obtain_feature_type_table(df):
    """"Function to predict the feature types
    INPUT: pandas DataFrame
    OUTPUT: Pandas DataFrame with feature types at index 0 and nothing else : [column names ]
                                                                              [feature_types]
    """
    #print('df.columns:', list(df.columns))
    predicted_feature_types = sortinghatinf.get_expanded_feature_types(df)
    feature_type_table = pd.DataFrame(columns=list(df.columns))
    feature_type_table.loc[len(feature_type_table)] = predicted_feature_types

    return feature_type_table


In [143]:
feature_type_table = obtain_feature_type_table(data)
print(list(feature_type_table.columns))
print(list(feature_type_table.iloc[0]))



['Unnamed: 0', 'Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Credit amount', 'Duration', 'Purpose']
['not-generalizable', 'integer', 'categorical', 'categorical', 'categorical', 'categorical', 'categorical', 'integer', 'integer', 'categorical']
